In [1]:
import os, sys, glob, shutil
import argparse
import json
import yaml
import numpy as np
from pprint import pprint

import torch
import yaml
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from scipy.io import wavfile
from matplotlib import pyplot as plt

In [2]:
# from utils.auto_tqdm import tqdm

In [3]:
%cd "D:\Schoolwork\TERM 3\WORK\visual_prosody"

D:\Schoolwork\TERM 3\WORK\visual_prosody


In [17]:
split_txt_val_path = r'.\preprocessed_data\Ego4D_final_v6\val.txt'
val_uids = []
with open(split_txt_val_path) as file:
    for line in file:
        # print(line.split('|')[0])
        val_uids.append(line.split('|')[0])

In [5]:
print(len(val_uids))

2772


In [6]:
from utils.model import get_model, get_vocoder, get_param_num, vocoder_infer
from utils.tools import to_device, log, synth_one_sample, expand, plot_mel
from model import FastSpeech2Loss
from dataset import Dataset
from utils.auto_tqdm import tqdm

from evaluate import evaluate

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
parser = argparse.ArgumentParser()
parser.add_argument("--restore_step", type=int, default=0)
parser.add_argument(
    "-p",
    "--preprocess_config",
    type=str,
    required=True,
    help="path to preprocess.yaml",
)
parser.add_argument(
    "-m", "--model_config", type=str, required=True, help="path to model.yaml"
)
parser.add_argument(
    "-t", "--train_config", type=str, required=True, help="path to train.yaml"
)

argString = '-p ./config/Ego4D_final_v6/0629b_preprocess.yaml -m ./config/Ego4D_final_v6/0629b_model.yaml -t ./config/Ego4D_final_v6/0629b_train.yaml'
# args = parser.parse_args()
args = parser.parse_args(argString.split())

In [9]:
pprint(args)

Namespace(restore_step=0, preprocess_config='./config/Ego4D_final_v6/0629b_preprocess.yaml', model_config='./config/Ego4D_final_v6/0629b_model.yaml', train_config='./config/Ego4D_final_v6/0629b_train.yaml')


In [10]:
# Read Config
preprocess_config = yaml.load(
    open(args.preprocess_config, "r"), Loader=yaml.FullLoader
)
model_config = yaml.load(open(args.model_config, "r"), Loader=yaml.FullLoader)
train_config = yaml.load(open(args.train_config, "r"), Loader=yaml.FullLoader)
configs = (preprocess_config, model_config, train_config)

In [11]:
print("Prepare training ...")

preprocess_config, model_config, train_config = configs

Prepare training ...


In [12]:
ckpt_path = r'./output/0629b/ckpt/Ego4D_final_v6/600000.pth.tar'
ckpt = torch.load(ckpt_path)

In [13]:
# Prepare model
model, optimizer = get_model(args, configs, device, train=True)
model.load_state_dict(ckpt["model"], strict=False)
model.to(device)
model = nn.DataParallel(model)
num_param = get_param_num(model)
Loss = FastSpeech2Loss(preprocess_config, model_config).to(device)
print("Number of FastSpeech2 Parameters:", num_param)

=> Not using speaker embeddings.
False
None
Number of FastSpeech2 Parameters: 35159745


In [14]:
# Load vocoder
vocoder = get_vocoder(model_config, device)

Removing weight norm...


In [15]:
step = args.restore_step + 1
model.eval()
print()

In [16]:
dataset = Dataset(
    "val.txt", 'val', preprocess_config, train_config, sort=False, drop_last=False
)

In [17]:
batch_size = train_config["optimizer"]["batch_size"]
batch_size = 1

In [18]:
loader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=False,
        collate_fn=dataset.collate_fn,
    )
# Get loss function
Loss = FastSpeech2Loss(preprocess_config, model_config).to(device)

In [19]:
os.path.join(train_config['path']['result_path'], 'plots')

'./output/0629b/result/Ego4D_final_v6\\plots'

In [20]:
# => batch:
# return (
#     ids,
#     raw_texts,
#     speakers,
#     texts,
#     text_lens,
#     max(text_lens),
#     mels,
#     mel_lens,
#     max(mel_lens),
#     pitches,
#     energies,
#     durations,
#     speaker_embeddings,
# )
# [12] 对应speaker embedding


# => model output / prediction
# return (
#     output,
#     postnet_output,
#     p_predictions,
#     e_predictions,
#     log_d_predictions,
#     d_rounded,
#     src_masks,
#     mel_masks,
#     src_lens,
#     mel_lens,
# )
output_plot_path = os.path.join(train_config['path']['result_path'], 'plot')
output_mel_syn_path = os.path.join(train_config['path']['result_path'], 'mel', 'syn')
output_mel_gt_path = os.path.join(train_config['path']['result_path'], 'mel', 'gt')
output_wav_syn_path = os.path.join(train_config['path']['result_path'], 'wav', 'synthesized')
output_wav_rec_path = os.path.join(train_config['path']['result_path'], 'wav', 'reconstructed')
os.makedirs(output_plot_path, exist_ok=True)
os.makedirs(output_mel_syn_path, exist_ok=True)
os.makedirs(output_mel_gt_path, exist_ok=True)
os.makedirs(output_wav_syn_path, exist_ok=True)
os.makedirs(output_wav_rec_path, exist_ok=True)

In [21]:
for batchs in tqdm(loader):
    for targets in batchs:
        targets = to_device(targets, device)
        with torch.no_grad():
            predictions = model(*(targets[2:]))
        basenames = targets[0]
        for i in range(len(predictions[0])):
            basename = basenames[i]
            src_len = predictions[8][i].item()
            mel_len = predictions[9][i].item()
            mel_prediction = predictions[1][i, :mel_len].detach().transpose(0, 1)
            mel_target = targets[6][i, :mel_len].detach().transpose(0, 1)

            torch.save(mel_prediction.cpu(), os.path.join(output_mel_syn_path, f"{basename}.pt"))
            torch.save(mel_target.cpu(), os.path.join(output_mel_gt_path, f"{basename}.pt"))
            
            
            duration = predictions[5][i, :src_len].detach().cpu().numpy()
            if preprocess_config["preprocessing"]["pitch"]["feature"] == "phoneme_level":
                pitch = predictions[2][i, :src_len].detach().cpu().numpy()
                pitch = expand(pitch, duration)
            else:
                pitch = predictions[2][i, :mel_len].detach().cpu().numpy()
            if preprocess_config["preprocessing"]["energy"]["feature"] == "phoneme_level":
                energy = predictions[3][i, :src_len].detach().cpu().numpy()
                energy = expand(energy, duration)
            else:
                energy = predictions[3][i, :mel_len].detach().cpu().numpy()

            with open(os.path.join(preprocess_config["path"]["preprocessed_path"], 
                                   "stats.json")) as f:
                stats = json.load(f)
                stats = stats["pitch"] + stats["energy"][:2]
                                       
            fig = plot_mel(
                [
                    (mel_prediction.cpu().numpy(), pitch, energy),
                    (mel_target.cpu().numpy(), pitch, energy),
                ],
                stats,
                ["Synthetized Spectrogram", "Ground-Truth Spectrogram"],
            )
            ### TODO: change to svg
            plt.savefig(os.path.join(output_plot_path, f"{basename}.png"))
            plt.close()

        # from .model import vocoder_infer

        mel_predictions = predictions[1].transpose(1, 2)
        mel_targets = targets[6].transpose(1, 2)
        
        lengths = predictions[9] * preprocess_config["preprocessing"]["stft"]["hop_length"]
        wav_predictions = vocoder_infer(
            mel_predictions, vocoder, model_config, preprocess_config, lengths=lengths
        )
        wav_targets = vocoder_infer(
        mel_targets, vocoder, model_config, preprocess_config, lengths=lengths
    )
    
        sampling_rate = preprocess_config["preprocessing"]["audio"]["sampling_rate"]
        for wav, basename in zip(wav_predictions, basenames):
            wavfile.write(os.path.join(output_wav_syn_path, f"{basename}.wav"), sampling_rate, wav)
        for wav, basename in zip(wav_targets, basenames):
            wavfile.write(os.path.join(output_wav_rec_path, f"{basename}.wav"), sampling_rate, wav)

        
    #     break
    # break

  0%|          | 0/2772 [00:00<?, ?it/s]

# Metrics

In [22]:
%cd "D:\Schoolwork\TERM 3\WORK\visual_prosody"

D:\Schoolwork\TERM 3\WORK\visual_prosody


In [4]:
import os, sys, glob, shutil
import argparse
import json
import yaml
import numpy as np
from pprint import pprint

In [5]:
import logging
import os
import os.path as op

import torch
import torch.nn.functional as F
import numpy as np
import torchaudio
import pandas as pd

In [6]:
from tqdm.notebook import tqdm

## MCD

In [26]:
def antidiag_indices(offset, min_i=0, max_i=None, min_j=0, max_j=None):
    """
    for a (3, 4) matrix with min_i=1, max_i=3, min_j=1, max_j=4, outputs
    offset=2 (1, 1),
    offset=3 (2, 1), (1, 2)
    offset=4 (2, 2), (1, 3)
    offset=5 (2, 3)
    constraints:
        i + j = offset
        min_j <= j < max_j
        min_i <= offset - j < max_i
    """
    if max_i is None:
        max_i = offset + 1
    if max_j is None:
        max_j = offset + 1
    min_j = max(min_j, offset - max_i + 1, 0)
    max_j = min(max_j, offset - min_i + 1, offset + 1)
    j = torch.arange(min_j, max_j)
    i = offset - j
    return torch.stack([i, j])

def batch_dynamic_time_warping(distance, shapes=None):
    """full batched DTW without any constraints
    distance:  (batchsize, max_M, max_N) matrix
    shapes: (batchsize,) vector specifying (M, N) for each entry
    """
    # ptr: 0=left, 1=up-left, 2=up
    ptr2dij = {0: (0, -1), 1: (-1, -1), 2: (-1, 0)}

    bsz, m, n = distance.size()
    cumdist = torch.zeros_like(distance)
    backptr = torch.zeros_like(distance).type(torch.int32) - 1

    # initialize
    cumdist[:, 0, :] = distance[:, 0, :].cumsum(dim=-1)
    cumdist[:, :, 0] = distance[:, :, 0].cumsum(dim=-1)
    backptr[:, 0, :] = 0
    backptr[:, :, 0] = 2

    # DP with optimized anti-diagonal parallelization, O(M+N) steps
    for offset in range(2, m + n - 1):
        ind = antidiag_indices(offset, 1, m, 1, n)
        c = torch.stack(
            [cumdist[:, ind[0], ind[1] - 1], cumdist[:, ind[0] - 1, ind[1] - 1],
             cumdist[:, ind[0] - 1, ind[1]], ],
            dim=2
        )
        v, b = c.min(axis=-1)
        backptr[:, ind[0], ind[1]] = b.int()
        cumdist[:, ind[0], ind[1]] = v + distance[:, ind[0], ind[1]]

    # backtrace
    pathmap = torch.zeros_like(backptr)
    for b in range(bsz):
        i = m - 1 if shapes is None else (shapes[b][0] - 1).item()
        j = n - 1 if shapes is None else (shapes[b][1] - 1).item()
        dtwpath = [(i, j)]
        while (i != 0 or j != 0) and len(dtwpath) < 10000:
            assert (i >= 0 and j >= 0)
            di, dj = ptr2dij[backptr[b, i, j].item()]
            i, j = i + di, j + dj
            dtwpath.append((i, j))
        dtwpath = dtwpath[::-1]
        indices = torch.from_numpy(np.array(dtwpath))
        pathmap[b, indices[:, 0], indices[:, 1]] = 1

    return cumdist, backptr, pathmap


def compute_l2_dist(x1, x2):
    """compute an (m, n) L2 distance matrix from (m, d) and (n, d) matrices"""
    return torch.cdist(x1.unsqueeze(0), x2.unsqueeze(0), p=2).squeeze(0).pow(2)


def compute_rms_dist(x1, x2):
    l2_dist = compute_l2_dist(x1, x2)
    return (l2_dist / x1.size(1)).pow(0.5)


def get_divisor(pathmap, normalize_type):
    if normalize_type is None:
        return 1
    elif normalize_type == "len1":
        return pathmap.size(0)
    elif normalize_type == "len2":
        return pathmap.size(1)
    elif normalize_type == "path":
        return pathmap.sum().item()
    else:
        raise ValueError(f"normalize_type {normalize_type} not supported")


In [27]:
def batch_mel_cepstral_distortion(
        y1, y2, sr, normalize_type="path", mfcc_fn=None
):
    # https://huggingface.co/spaces/OFA-Sys/OFA-Generic_Interface/blob/main/fairseq/fairseq/tasks/text_to_speech.py
    """
    https://arxiv.org/pdf/2011.03568.pdf
    The root mean squared error computed on 13-dimensional MFCC using DTW for
    alignment. MFCC features are computed from an 80-channel log-mel
    spectrogram using a 50ms Hann window and hop of 12.5ms.
    y1: list of waveforms
    y2: list of waveforms
    sr: sampling rate
    """

    try:
        import torchaudio
    except ImportError:
        raise ImportError("Please install torchaudio: pip install torchaudio")

    if mfcc_fn is None or mfcc_fn.sample_rate != sr:
        melkwargs = {
            "n_fft": int(0.05 * sr), "win_length": int(0.05 * sr),
            "hop_length": int(0.0125 * sr), "f_min": 20,
            "n_mels": 80, "window_fn": torch.hann_window
        }
        mfcc_fn = torchaudio.transforms.MFCC(
            sr, n_mfcc=13, log_mels=True, melkwargs=melkwargs
        ).to(y1[0].device)
    return batch_compute_distortion(
        y1, y2, sr, lambda y: mfcc_fn(y).transpose(-1, -2), compute_rms_dist,
        normalize_type
    )

    
def batch_compute_distortion(y1, y2, sr, feat_fn, dist_fn, normalize_type):
    d, s, x1, x2 = [], [], [], []
    for cur_y1, cur_y2 in zip(y1, y2):
        assert (cur_y1.ndim == 1 and cur_y2.ndim == 1)
        cur_x1 = feat_fn(cur_y1)
        cur_x2 = feat_fn(cur_y2)
        x1.append(cur_x1)
        x2.append(cur_x2)

        cur_d = dist_fn(cur_x1, cur_x2)
        d.append(cur_d)
        s.append(d[-1].size())
    max_m = max(ss[0] for ss in s)
    max_n = max(ss[1] for ss in s)
    d = torch.stack(
        [F.pad(dd, (0, max_n - dd.size(1), 0, max_m - dd.size(0))) for dd in d]
    )
    s = torch.LongTensor(s).to(d.device)
    cumdists, backptrs, pathmaps = batch_dynamic_time_warping(d, s)

    rets = []
    itr = zip(s, x1, x2, d, cumdists, backptrs, pathmaps)
    for (m, n), cur_x1, cur_x2, dist, cumdist, backptr, pathmap in itr:
        cumdist = cumdist[:m, :n]
        backptr = backptr[:m, :n]
        pathmap = pathmap[:m, :n]
        divisor = get_divisor(pathmap, normalize_type)

        distortion = cumdist[-1, -1] / divisor
        ret = distortion, (cur_x1, cur_x2, dist, cumdist, backptr, pathmap)
        rets.append(ret)
    return rets

In [28]:
ref_folder = r'.\output\0629b\result\Ego4D_final_v6\wav\reconstructed'
syn_folder = r'.\output\0629b\result\Ego4D_final_v6\wav\synthesized'

In [29]:
split_txt_val_path = r'.\preprocessed_data\Ego4D_final_v6\val.txt'
val_uids = []
with open(split_txt_val_path) as file:
    for line in file:
        # print(line.split('|')[0])
        val_uids.append(line.split('|')[0])

In [30]:
len(val_uids)

2772

In [31]:
uids = []
MCDs = []

In [32]:
for uid in tqdm(val_uids):
    uids.append(uid)
    wav_1, sr = torchaudio.load(op.join(ref_folder, f"{uid}.wav"))
    wav_2, sr = torchaudio.load(op.join(syn_folder, f"{uid}.wav"))
    wav_1 = wav_1.squeeze()
    wav_2 = wav_2.squeeze()
    results = batch_mel_cepstral_distortion(
            [wav_1],
            [wav_2],
            sr=sr,
    )
    MCDs.append(results[0][0].item())

  0%|          | 0/2772 [00:00<?, ?it/s]

In [33]:
import pandas as pd

lj_mcd_df = pd.DataFrame({
    'uid': uids,
    'MCD': MCDs,
})

In [34]:
lj_mcd_df.to_csv(r".\jupyter_walkthrough\metrics\MCD_0629b.csv")

In [35]:
print(f"MCD mean on 0629b: {torch.tensor(MCDs).mean()}")
print(f"MCD std on 0629b: {torch.tensor(MCDs).std()}")

MCD mean on 0629b: 4.545107364654541
MCD std on 0629b: 1.0436636209487915


In [7]:
import pyworld as pw
import matplotlib.pyplot as plt

In [8]:
%cd "D:\Schoolwork\TERM 3\WORK\visual_prosody"

D:\Schoolwork\TERM 3\WORK\visual_prosody


In [9]:
import os, sys, glob, shutil
import argparse
import json
import yaml
import numpy as np
from pprint import pprint
import logging
import os
import os.path as op

import torch
import torch.nn.functional as F
import numpy as np
import torchaudio
import pandas as pd

from tqdm.notebook import tqdm

In [10]:
import argparse
import fnmatch
import logging
import multiprocessing as mp
import os
from typing import Dict, List, Tuple

import librosa
import numpy as np
import pysptk
import pyworld as pw
import soundfile as sf
from fastdtw import fastdtw
from scipy import spatial

In [11]:
def world_extract(
    x: np.ndarray,
    fs: int,
    f0min: int = 40,
    f0max: int = 800,
    n_fft: int = 512,
    n_shift: int = 256,
    mcep_dim: int = 13,
    mcep_alpha: float = 0.41,
) -> np.ndarray:
    """Extract World-based acoustic features.

    Args:
        x (ndarray): 1D waveform array.
        fs (int): Minimum f0 value (default=40).
        f0 (int): Maximum f0 value (default=800).
        n_shift (int): Shift length in point (default=256).
        n_fft (int): FFT length in point (default=512).
        n_shift (int): Shift length in point (default=256).
        mcep_dim (int): Dimension of mel-cepstrum (default=25).
        mcep_alpha (float): All pass filter coefficient (default=0.41).

    Returns:
        ndarray: Mel-cepstrum with the size (N, n_fft).
        ndarray: F0 sequence (N,).

    """
    # extract features
    x = x.astype(np.float64)
    # f0, time_axis = pw.harvest(
    #     x,
    #     fs,
    #     f0_floor=f0min,
    #     f0_ceil=f0max,
    #     frame_period=n_shift / fs * 1000,
    # )
    pitch, t = pw.dio(
        x,
        fs,
         frame_period=n_shift / fs * 1000,
        )
    sp = pw.cheaptrick(x, f0, time_axis, fs, fft_size=n_fft)
    if mcep_dim is None or mcep_alpha is None:
        mcep_dim, mcep_alpha = _get_best_mcep_params(fs)
    mcep = pysptk.sp2mc(sp, 13, mcep_alpha)

    return mcep, f0


def _get_basename(path: str) -> str:
    return os.path.splitext(os.path.split(path)[-1])[0]


def _get_best_mcep_params(fs: int) -> Tuple[int, float]:
    if fs == 16000:
        return 23, 0.42
    elif fs == 22050:
        return 34, 0.45
    elif fs == 24000:
        return 34, 0.46
    elif fs == 44100:
        return 39, 0.53
    elif fs == 48000:
        return 39, 0.55
    else:
        raise ValueError(f"Not found the setting for {fs}.")

In [12]:
ref_folder = r'.\output\0629b\result\Ego4D_final_v6\wav\reconstructed'
syn_folder = r'.\output\0629b\result\Ego4D_final_v6\wav\synthesized'

In [56]:
split_txt_val_path = r'.\preprocessed_data\Ego4D_final_v6\val.txt'
val_uids = []
with open(split_txt_val_path) as file:
    for line in file:
        # print(line.split('|')[0])
        val_uids.append(line.split('|')[0])

In [57]:
sr = 22050
melkwargs = {
    "n_fft": int(0.05 * sr), "win_length": int(0.05 * sr),
    "hop_length": int(0.0125 * sr), "f_min": 20,
    "n_mels": 80, "window_fn": torch.hann_window
}
mfcc_fn = torchaudio.transforms.MFCC(
    sr, n_mfcc=13, log_mels=True, melkwargs=melkwargs
)

In [59]:
uids = []
MCDs = []

In [60]:
for uid in tqdm(val_uids):
    uids.append(uid)
    wav_1, sr = torchaudio.load(op.join(ref_folder, f"{uid}.wav"))
    wav_2, sr = torchaudio.load(op.join(syn_folder, f"{uid}.wav"))
    wav_1 = wav_1.squeeze()
    wav_2 = wav_2.squeeze()
    mel_1 = mfcc_fn(wav_1).T.numpy()
    mel_2 = mfcc_fn(wav_2).T.numpy()
    # DTW
    _, path = fastdtw(mel_2, mel_1, dist=spatial.distance.euclidean)
    twf = np.array(path).T
    mel_2 = mel_2[twf[0]]
    mel_1 = mel_1[twf[1]]
    # We sum the squared differences over the first K MFCCs, skipping ct,0
    mel_1 = mel_1[:, 1:]
    mel_2 = mel_2[:, 1:]
    result = (((mel_1 - mel_2) ** 2).sum(axis=1)**0.5).mean()
    MCDs.append(result)

  0%|          | 0/2772 [00:00<?, ?it/s]

In [61]:
import pandas as pd

lj_mcd_df = pd.DataFrame({
    'uid': uids,
    'MCD': MCDs,
})

In [62]:
lj_mcd_df.to_csv(r".\jupyter_walkthrough\metrics\MCD_0629b.csv")

In [63]:
print(f"MCD mean on 0629b: {torch.tensor(MCDs).mean()}")
print(f"MCD std on 0629b: {torch.tensor(MCDs).std()}")

MCD mean on 0629b: 13.024544715881348
MCD std on 0629b: 2.407449960708618


## log f0

In [13]:
# https://github.com/espnet/espnet/blob/3e0dad524d62ccd45e067e9b36049f2214ea972a/egs2/TEMPLATE/asr1/pyscripts/utils/evaluate_f0.py

In [14]:
def world_extract(
    x: np.ndarray,
    fs: int,
    f0min: int = 40,
    f0max: int = 800,
    n_fft: int = 512,
    n_shift: int = 256,
    mcep_dim: int = 25,
    mcep_alpha: float = 0.41,
) -> np.ndarray:
    """Extract World-based acoustic features.

    Args:
        x (ndarray): 1D waveform array.
        fs (int): Minimum f0 value (default=40).
        f0 (int): Maximum f0 value (default=800).
        n_shift (int): Shift length in point (default=256).
        n_fft (int): FFT length in point (default=512).
        n_shift (int): Shift length in point (default=256).
        mcep_dim (int): Dimension of mel-cepstrum (default=25).
        mcep_alpha (float): All pass filter coefficient (default=0.41).

    Returns:
        ndarray: Mel-cepstrum with the size (N, n_fft).
        ndarray: F0 sequence (N,).

    """
    # extract features
    x = x.astype(np.float64)
    f0, time_axis = pw.harvest(
        x,
        fs,
        f0_floor=f0min,
        f0_ceil=f0max,
        frame_period=n_shift / fs * 1000,
    )
    sp = pw.cheaptrick(x, f0, time_axis, fs, fft_size=n_fft)
    if mcep_dim is None or mcep_alpha is None:
        mcep_dim, mcep_alpha = _get_best_mcep_params(fs)
    mcep = pysptk.sp2mc(sp, mcep_dim, mcep_alpha)

    return mcep, f0


def _get_basename(path: str) -> str:
    return os.path.splitext(os.path.split(path)[-1])[0]


def _get_best_mcep_params(fs: int) -> Tuple[int, float]:
    if fs == 16000:
        return 23, 0.42
    elif fs == 22050:
        return 34, 0.45
    elif fs == 24000:
        return 34, 0.46
    elif fs == 44100:
        return 39, 0.53
    elif fs == 48000:
        return 39, 0.55
    else:
        raise ValueError(f"Not found the setting for {fs}.")

In [24]:
uids = []
logf0_rmses = []
logf0_corrs = []

In [25]:
for uid in tqdm(val_uids):
    
    # load wav file as int16
    gen_x, gen_fs = sf.read(op.join(syn_folder, f"{uid}.wav"), dtype="int16")
    gt_x, gt_fs = sf.read(op.join(ref_folder, f"{uid}.wav"), dtype="int16")
    fs = gen_fs
    # extract ground truth and converted features
    gen_mcep, gen_f0 = world_extract(
        x=gen_x,
        fs=fs,
        f0min=40,
        f0max=800,
        n_fft=1024,
        n_shift=256,
        mcep_dim=None,
        mcep_alpha=None,
    )
    gt_mcep, gt_f0 = world_extract(
        x=gt_x,
        fs=fs,
        f0min=40,
        f0max=800,
        n_fft=1024,
        n_shift=256,
        mcep_dim=None,
        mcep_alpha=None,
    )
    
    # DTW
    _, path = fastdtw(gen_mcep, gt_mcep, dist=spatial.distance.euclidean)
    twf = np.array(path).T
    gen_f0_dtw = gen_f0[twf[0]]
    gt_f0_dtw = gt_f0[twf[1]]
    
    # Get voiced part
    nonzero_idxs = np.where((gen_f0_dtw != 0) & (gt_f0_dtw != 0))[0]
    eps = 1e-7
    gen_f0_dtw_voiced = np.log(gen_f0_dtw[nonzero_idxs] + eps)
    gt_f0_dtw_voiced = np.log(gt_f0_dtw[nonzero_idxs] + eps)

    if len(gen_f0_dtw_voiced) == 0 or len(gt_f0_dtw_voiced) == 0:
        print(f"Skip uid {uid}. len == 0.")
        continue

    # log F0 RMSE
    log_f0_rmse = np.sqrt(np.mean((gen_f0_dtw_voiced - gt_f0_dtw_voiced) ** 2))
    # print(f"{uid} {log_f0_rmse:.4f}")

    # log F0 corr
    log_f0_corr = np.corrcoef(gen_f0_dtw_voiced, gt_f0_dtw_voiced)[0][1]
    # print(f"{uid} {log_f0_corr:.4f}")
    uids.append(uid)
    logf0_rmses.append(log_f0_rmse)
    logf0_corrs.append(log_f0_corr)

  0%|          | 0/2772 [00:00<?, ?it/s]

Skip uid 342edcf2-1150-42a0-af79-030571c0996d. len == 0.
Skip uid fb487f40-31b3-4397-a656-5099380efa0b. len == 0.
Skip uid f03be5e0-c2b2-4479-9b4f-30730e9e30a6. len == 0.
Skip uid 2ff974ac-b0f1-4611-8c9f-61c216a8ba96. len == 0.
Skip uid 67ae347b-a4a6-4966-a736-c03a831d1a2a. len == 0.
Skip uid 959dcb2b-4291-45b4-aa24-dc144de6fc1a. len == 0.
Skip uid 94ab3c42-86a9-427c-9dc7-9d3583836f89. len == 0.
Skip uid 3c82b212-ca78-4648-8342-56c8baa097b3. len == 0.
Skip uid 1d22ffa5-f5bb-4f58-a466-b5ea864af573. len == 0.
Skip uid f47ab348-daa9-4497-8abf-75ff2ca41285. len == 0.
Skip uid 2eae0b32-1925-4045-9c7e-b35cbf0273ad. len == 0.
Skip uid 8b91e519-e0b5-43b6-a809-652fff3351e9. len == 0.
Skip uid 9f56d2e9-8536-4ce4-878e-e31f5c64345a. len == 0.
Skip uid 06c43857-25aa-4c20-88ea-bb75150a2b55. len == 0.
Skip uid 346a8e85-6aff-43d9-87c1-5884a4466686. len == 0.
Skip uid 0d885f9d-8786-40d2-ab4b-d6874b5c6507. len == 0.
Skip uid ffc09302-1833-46b3-a42b-0e4a8c52906f. len == 0.
Skip uid 62a8c03b-e5ce-494a-bd8

In [26]:
lj_logf0_df = pd.DataFrame({
    'uid': uids,
    'logf0_rmse': logf0_rmses,
    'logf0_corr': logf0_corrs,
})
lj_logf0_df.to_csv(r".\jupyter_walkthrough\metrics\logF0_0629b.csv")

In [32]:
lj_logf0_df.shape

(2751, 3)

In [34]:
lj_logf0_df.dropna().shape

(2746, 3)

In [35]:
lj_logf0_df = lj_logf0_df.dropna()

In [40]:
lj_logf0_df

,uid,logf0_rmse,logf0_corr
0,9d58583c-20de-439d-b1cd-9c2265bdedd8,0.181933,-0.200166
1,4a506707-52ea-493e-98c4-f667e3222d44,0.368843,-0.594507
2,0b67f942-6ebf-403d-aff0-f4e5d62d3140,0.128017,0.742773
3,c237c6d5-f413-4e25-82e2-af7b408d390a,0.301658,-0.167710
4,b8ed5cf9-b8ff-4da3-a764-a59020b85277,0.497127,0.149655
...,...,...,...
2746,c1f709a3-3952-4f3f-9381-3a97d2b303bb,0.694469,0.209543
2747,b84c4dbf-c88a-43da-a5cb-aa2ced16e17d,0.827369,0.092270
2748,c7a54b50-6ca6-49cd-b048-abe1770b19c9,0.675416,0.182086
2749,a0c002a1-ce41-49bb-af4a-6df4dae2e2cd,0.330229,0.315930


In [41]:
lj_logf0_df['logf0_rmse'].values
lj_logf0_df['logf0_corr'].values

array([-0.20016617, -0.59450695,  0.74277257, ...,  0.18208585,
        0.31592969,  0.49965225])

In [42]:
print(f"logf0_rmse mean on 0629b: {torch.tensor(lj_logf0_df['logf0_rmse'].values).mean()}")
print(f"logf0_rmse std on 0629b: {torch.tensor(lj_logf0_df['logf0_rmse'].values).std()}")

print(f"logf0_corr mean on 0629b: {torch.tensor(lj_logf0_df['logf0_corr'].values).mean()}")
print(f"logf0_corr std on 0629b: {torch.tensor(lj_logf0_df['logf0_corr'].values).std()}")

logf0_rmse mean on 0629b: 0.347476650443757
logf0_rmse std on 0629b: 0.27812424555593845
logf0_corr mean on 0629b: 0.30949771492752826
logf0_corr std on 0629b: 0.4162518243168433
